# The One Time Pad cipher
_Recap: A cipher is a technique that can encrypt or decrypt data (which is commonly referred to as "plaintext" or "cleartext"). During encryption, the input plaintext is converted into what is known as the "ciphertext". It is impossible to read or extract any meaning from the ciphertext unless you happen to know how the cipher works, and also a special input to the cipher known as the "key". The same cipher, with the same key, can be used to decrypt the ciphertext back into plaintext to make it readable again._

The One Time Pad (OTP) is a very simple cipher where the cleartext is bitwise XORed with the key to form the ciphertext. When the same key is used to decrypt the ciphertext, we get back the plaintext. XOR is one of the most fundamental logic operations that takes as input two bits, and outputs one bit. The truth table is:

| A | B | Y = A XOR B |
| - | - | :-----------: |
| 0 | 0 | 0 |
| 0 | 1 | 1 |
| 1 | 0 | 1 |
| 1 | 1 | 0 |

In Python, the symbol for the XOR operation is `^`.

In [ ]:
print(0^0)
print(0^1)
print(1^0)
print(1^1)

## Encryption

Let us use the BitVector library to implement the One Time Pad cipher. First, we need to choose a plaintext, and convert it to bits using an encoding scheme.

In [ ]:
from BitVector import BitVector
plaintext = BitVector(textstring="hello")
print(plaintext)
print(plaintext.get_hex_string_from_bitvector())

Next, we will need a key. Because of how OTP works, the key needs to be the same length as the plaintext.

In [ ]:
otp_key = BitVector(size=plaintext.size)
otp_key = otp_key.gen_random_bits(otp_key.size)
print(otp_key)
print(otp_key.get_hex_string_from_bitvector())

We are now ready to encrypt the plaintext. All we have to do is XOR the two BitVectors!

In [ ]:
ciphertext = plaintext ^ otp_key
print(ciphertext)
print(ciphertext.get_hex_string_from_bitvector())
print(ciphertext.get_bitvector_in_ascii())

As we can see, the ciphertext is a jumble of symbols!

## Decryption

Decryption is equally simple - we just XOR the *same key* with the ciphertext!

In [ ]:
decrypted = ciphertext ^ otp_key
print(decrypted)
print(decrypted.get_hex_string_from_bitvector())
print(decrypted.get_bitvector_in_ascii())

We get back the original plaintext! What happens when an adversary Trudy tries to decrypt the ciphertext? Because the key is not known to anyone else other than Alice and Bob, Trudy has to guess the key, which means that she chooses a random set of bits as the key.

In [ ]:
trudy_key = BitVector(size=ciphertext.size)
trudy_key = trudy_key.gen_random_bits(trudy_key.size)

trudy_plaintext = ciphertext ^ trudy_key
print(trudy_plaintext)
print(trudy_plaintext.get_hex_string_from_bitvector())
print(trudy_plaintext.get_bitvector_in_ascii())

Trudy is unable to get the original plaintext! How difficult is it for Trudy to guess the key? What are the chances? It is `1` divided by the number of possible keys, which in turn is two (because it's in binary) raised to the length of the key in bits. To find out:

In [ ]:
size = trudy_key.size
tot_keys = pow(2, size)
chance = 1/tot_keys

print(size, tot_keys, chance)

We see that for just five ASCII characters, the number of bits is `8*5=40`, which means that the chance that Trudy can guess the key correctly is about one in one trillion! More importantly, there is *no other way* that Trudy can improve her chances of guessing the correct key. OTP is indeed a very strong cipher, provided that the key is not reused.

This concludes the tutorial. Feel free to modify and extend the above code. For example, you could extend the above code to:

* use another logical operation such as AND, NAND or OR, and see what happens
* brute force the decryption key by trying all possible keys (may take a really long time to run, so watch out!)

Try it out!

# What happens when you try to brute-force (decrypt) the ciphertext?

In this series of exercises, we will try to decrypt a OTP-encrypted ciphertext. We're going to try all possible keys, and apply each key to the ciphertext in the hope that it will yield a readable, valid plaintext. We also need a way to filter out unreadable outputs - we'll do this by looking up each output in a dictionary of valid words. If the list contains the word, we retain it, else we will discard it.

## Exercise 1: read a word list from a file

The file `scrabble.txt` contains a list of words accepted in the game Scrabble. Each word is on its own line, in uppercase. Your first task is to read each line from a file, strip the newline `\n` character, convert them to lowercase, and store the words in a list for later use. To keep the list small, we're only going to work with two-letter words (and there are quite a few of them!).

Here, each line in the file is being read into the `line` variable after opening the file. To remove newline/space characters from a string, use the `.rstrip()` function on the variable like this: `line.rstrip()`. If you don't do this, then the length of string will be higher since a newline also counts as a character.

To calculate the length of a string use the `len()` function like this: `len(line)`. To convert a string to lowercase, use the `.lower()` function just like `.rstrip()`. You can chain functions too: `line.rstrip().lower()` first removes unwanted characters and then converts it to lowercase.

Fill in the blanks in this code block:

In [ ]:
wordlist = []
for line in open("_______________"):
    if _________ <= ____: # calculate the length of the string and compare it here
        wordlist.append(___________) # convert the string to lowercase here


## Exercise 2: setup the brute force loop

Now that we have the word list setup, we're going to set up a loop to brute force decrypt our ciphertext `0x8816`. First, let's create a BitVector object, and initialize it with the ciphertext value. Let's create a BitVector object using the `hexstring` argument:

In [ ]:
ctext_bv = BitVector(hexstring="_____") # omit the 0x prefix here


The next step is to create another BV object to represent the key that we're going to brute force. It is important to set the size of the object (in bits) to be equal to the length of the ciphertext. It's useful to store the size as a variable so that we can reuse it later. Fill in the blanks in the below code:


In [ ]:
ctext_length = ________
key_bv = BitVector(size=__________)


Next, let's setup a loop to iterate over all possible values of the key. We do this by assigning a new value to the BV in every iteration. How many values do we need to iterate over? It's simply the number of possible values a bit vector of length `ctext_length` can take on, which is $2$ raised to `ctext_length`. Since we're working with base 10 here while calculating powers, we need to use the `intVal` argument to set the BV's value.

Hint: you can use the `pow(a,b)` function in Python to calculate $a^b$

In [ ]:
for i in range(0, ______):
    key_bv.set_value(intVal=_______, size=_______)

Now, you need to modify the above loop by adding functionality to it. Once you have set the value of the BV, you should use it to decrypt the ciphertext using the XOR operator `^`. Then, you should find the ASCII value of the BV using the `.get_bitvector_in_ascii()` method on the BV.

Next, you should look up the resulting ASCII value in the word list to see if it's a valid word. To do this, use an `if` condition: `if word in wordlist:` and on the next line you can print the word `print(word)`.

Once you've done this, it's time to start the loop and let it print all possible valid plaintext words corresponding to the ciphertext `0x8816`. How many valid words did you get?

## Exercise 3: double (successive) encryption using OTP

We've seen that it's fairly easy to brute force a two letter plaintext, even though we have multiple possibilities for the plaintext. How can we increase the time taken (by Trudy) to brute force without increasing the size of the ciphertext? It's simple: Alice or Bob can encrypt the plaintext _twice_ (or multiple times) using different keys.

Now, extend the above code you've written to brute force a double encrypted two letter plaintext. Use the same ciphertext as above. Try to run the loop. How long does it take to run? Is it twice as long as the above loop? Or longer?